In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, VBox, HBox, IntSlider, FloatSlider, Dropdown, Output, interactive
from IPython.display import display, clear_output
import scipy.signal
from math import log2, erfc, sqrt
import random

# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))
    
    return t,h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise


def interactive_signal_processing_with_natural(n_bits=80000, roll_off=0.7, nsamp=16, delay=4, L=16):
    # Common setup
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)

    # Gray encoding setup
    gray_encoding = grayCode(k)
    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]

    # Natural encoding setup
    natural_encoding = naturalBinaryCoding(k)
    x_natural = [y_levels[natural_encoding.index(symbol)] for symbol in symbols]

    # Root raised cosine filter (common for both Gray and Natural)
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)

    # Transmission simulation for Gray encoding
    y_gray = upSample(x_gray, nsamp)
    y_transmitted_gray = scipy.signal.convolve(y_gray, filt)

    # Transmission simulation for Natural encoding
    y_natural = upSample(x_natural, nsamp)
    y_transmitted_natural = scipy.signal.convolve(y_natural, filt)

    # Calculate BER for Gray encoding
    berTheoretical = []
    berSimulation_gray = []
    berSimulation_natural = [] # For natural encoding
    EbN0_max = 20

    for EbN0_db in range(1, EbN0_max):
        # Convert Eb/N0 from dB to linear scale
        EbN0 = 10 ** (EbN0_db * 0.1)
        SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
        SNR = 10 ** (SNR_db * 0.1)

        # Calculate the power of the transmitted signal and the noise power to achieve the desired SNR
        P = sum(y_transmitted_gray * y_transmitted_gray) / len(y_transmitted_gray)
        P_db = 10 * np.log10(P)
        Pn_db = P_db - SNR_db
        Pn = 10 ** (Pn_db * 0.1)

        # Add noise (common for both Gray and Natural)
        mu = 0
        sigma = np.sqrt(Pn)
        noise = generateAWGN(y_transmitted_gray, mu, sigma)
        y_noisy_gray = y_transmitted_gray + noise
        y_noisy_natural = y_transmitted_natural + noise

        # Receiver simulation for Gray encoding
        z_received_gray = scipy.signal.convolve(y_noisy_gray, filt)
        z_final_gray = downSample(z_received_gray, nsamp)
        z_final_gray = z_final_gray[2 * delay : len(z_final_gray) - 2 * delay]

        # Receiver simulation for Natural encoding (Code 1)
        z_received_natural = scipy.signal.convolve(y_noisy_natural, filt)
        z_final_natural = downSample(z_received_natural, nsamp)
        z_final_natural = z_final_natural[2 * delay : len(z_final_natural) - 2 * delay]

        # Error calculation for Gray encoding
        # Decision for the level corresponding to the symbol received for Gray encoding
        for i in range(len(z_final_gray)):
            # Array with the differences of the signal from the levels
            differences = np.abs(y_levels - z_final_gray[i]) 
            m = min(differences) # Find the minimum distance
            [index], = np.where(differences == m)
            z_final_gray[i] = y_levels[index]
        
        # We assume that each error in Gray encoding results in one incorrect bit
        error = 0
        for i in range(len(z_final_gray)):
            if x_gray[i] != z_final_gray[i]:
                error += 1
        
        # BER calculation = number of errors / number of bits
        berSimulation_gray.append(error/n_bits)

        # Error calculation for Natural encoding 
        # Decision for the level corresponding to the symbol received
        for i in range(len(z_final_natural)):
            differences = np.abs(y_levels - z_final_natural[i])
            m = min(differences)
            [index], = np.where(differences == m)
            z_final_natural[i] = y_levels[index]
        
        # The final_symbols contains the bits that were decided to be received
        final_symbols = []
        for i in range(len(z_final_natural)):
            # Mapping each decided level to its binary encoding
            index = y_levels.index(z_final_natural[i])
            final_symbols.append(natural_encoding[index])

        # For each incorrect symbol, we need to check how many bits were wrong,
        # as the encoding of neighboring levels no longer differs by only one bit 
        error = 0
        for i in range(len(z_final_natural)):
            # If a symbol is wrong, check how many digits were received incorrectly
            if x_natural[i] != z_final_natural[i]:
                for j in range(len(symbols[i])):
                    if symbols[i][j] != final_symbols[i][j]:
                        error += 1

        berSimulation_natural.append(error/n_bits)
        # Theoretical BER calculation (common for both Gray and Natural)
        Pe = (L - 1) / L * erfc(sqrt(3 * k / (L**2 - 1) * EbN0))
        berTheoretical.append(Pe / k)

    # Plotting the BER for both Gray and Natural encoding
    plt.figure(figsize=(10, 6))
    plt.semilogy(range(1, EbN0_max), berTheoretical, label='Theoretical')
    plt.semilogy(range(1, EbN0_max), berSimulation_gray, 'o', label='Gray')
    plt.semilogy(range(1, EbN0_max), berSimulation_natural, '*', label='Natural')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Rate')
    plt.title('BER Curve for L-ASK')
    plt.legend()
    plt.grid(True)

    plt.show()


# Creating sliders and dropdown
n_bits_slider = IntSlider(min=10000, max=100000, step=10000, value=80000, description='Binary Sequence Length', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
roll_off_slider = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Roll-off Factor', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
nsamp_slider = IntSlider(min=8, max=32, step=1, value=16, description='nsamp', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
delay_slider = IntSlider(min=1, max=8, step=1, value=4, description='Group Delay', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
L_dropdown = Dropdown(options=[2**i for i in range(1, 6)], value=16, description='ASK Levels', style={'description_width': 'initial'}, continuous_update=False)

# Reordering the widgets
interactive_plot_with_natural = interactive(interactive_signal_processing_with_natural, n_bits=n_bits_slider, roll_off=roll_off_slider, nsamp=nsamp_slider, delay=delay_slider, L=L_dropdown)

input_widgets = VBox([n_bits_slider, roll_off_slider, nsamp_slider, delay_slider, L_dropdown], layout=Layout(flex='1 1 auto', width='auto'))
plot_output = interactive_plot_with_natural.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets])

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)